In [7]:
import cv2
import numpy as np
import os

In [9]:
def enhance_image(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab[:, :, 0] = clahe.apply(lab[:, :, 0])
    enhanced_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    gamma = 1.2
    look_up_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(0, 256)]).astype("uint8")
    enhanced_image = cv2.LUT(enhanced_image, look_up_table)
    
    return enhanced_image


In [10]:
def segment_facial_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    mask = np.zeros_like(image)
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w]
            mask[y:y+h, x:x+w] = face
    else:
        mask = image.copy()
    return mask


In [11]:
def detect_edges(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    return edges


In [12]:
def blur_non_essential(image):
    blurred_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    return blurred_image


In [13]:
input_dir = r"C:\Users\User\OneDrive\Documents\Computer vision with DL\image processing exam\archive\train"
output_dir = r"C:\Users\User\OneDrive\Documents\Computer vision with DL\image processing exam\Output_Image_problem 2"

categories = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
tasks = ['enhanced', 'segmented', 'edges', 'blurred']

for category in categories:
    category_output_dir = os.path.join(output_dir, category)
    
    for task in tasks:
        task_output_dir = os.path.join(category_output_dir, task)
        os.makedirs(task_output_dir, exist_ok=True)

    input_category_dir = os.path.join(input_dir, category)

    for img_file in os.listdir(input_category_dir):
        if img_file.endswith(".png") or img_file.endswith(".jpg"):
            image_path = os.path.join(input_category_dir, img_file)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error loading image at path: {image_path}")
                continue

            enhanced_image = enhance_image(image)
            enhanced_path = os.path.join(category_output_dir, "enhanced", img_file)
            cv2.imwrite(enhanced_path, enhanced_image)

            segmented_image = segment_facial_features(enhanced_image)
            segmented_path = os.path.join(category_output_dir, "segmented", img_file)
            cv2.imwrite(segmented_path, segmented_image)

            edges_image = detect_edges(segmented_image)
            edges_path = os.path.join(category_output_dir, "edges", img_file)
            cv2.imwrite(edges_path, edges_image)

            blurred_image = blur_non_essential(enhanced_image)
            blurred_path = os.path.join(category_output_dir, "blurred", img_file)
            cv2.imwrite(blurred_path, blurred_image)

            print(f"Processed {img_file} in category {category} and saved outputs to task-specific directories")


Processed Training_10118481.jpg in category angry and saved outputs to task-specific directories
Processed Training_10120469.jpg in category angry and saved outputs to task-specific directories
Processed Training_10131352.jpg in category angry and saved outputs to task-specific directories
Processed Training_10161559.jpg in category angry and saved outputs to task-specific directories
Processed Training_1021836.jpg in category angry and saved outputs to task-specific directories
Processed Training_10269675.jpg in category angry and saved outputs to task-specific directories
Processed Training_10278738.jpg in category angry and saved outputs to task-specific directories
Processed Training_10290703.jpg in category angry and saved outputs to task-specific directories
Processed Training_10295477.jpg in category angry and saved outputs to task-specific directories
Processed Training_10315441.jpg in category angry and saved outputs to task-specific directories
Processed Training_10316849.jpg